In [63]:
import psycopg2

In [64]:
# Update connection string information obtained from the portal
host = "wangandyoondatabase.postgres.database.azure.com"
user = "mylogin@wangandyoondatabase"
dbname = "postgres"
password = "shampoo1!"
sslmode = "require"

In [65]:
# Construct connection string
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")

Connection established


In [66]:
cursor=conn.cursor()


In [67]:
import psycopg2
# Update connection string information obtained from the portal
host = "wangandyoondatabase.postgres.database.azure.com"
user = "mylogin@wangandyoondatabase"
dbname = "postgres"
password = "shampoo1!"
sslmode = "require"

# Construct connection string
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string)
print("Connection established")

cursor=conn.cursor()

cursor.execute("CREATE TABLE SentimentAnalysisTable (id serial PRIMARY KEY, UserID VARCHAR(50), Sentiment VARCHAR(10), Date VARCHAR(50) );")
print("Finished creating table")

from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer as Vader

# consumer key, consumer secret, access token, access secret.
ckey = "DWP7ziCeLmIF7EM6E3U1x4Awk"
csecret = "8AbR2jazy2FBNMu8d4Yyma6YPieSaEZniNBvCWngaN40BvTAO9"
atoken = "959357240787349505-xwLcOZ5LoVMXsLwlVzHcx3eHAOyB6SS"
asecret = "XfXLhs7dC7tHb49PmfbTkSFLoWBYoFmInFC6uMG5aLalS"


class listener(StreamListener):

    def on_data(self, data):
        Data = json.loads(data)

        if 'RT @' not in Data['text']:

            tweet = Data["text"]
            ID=Data["user"]["name"]
            Date=Data["created_at"]
            sid = Vader()
            ss = sid.polarity_scores(tweet)
            if ss['pos'] > 0 and ss['compound'] > 0:
                sentimentalStatus = 'Positive'
            elif ss['neg'] > 0 and ss['compound'] < 0:
                sentimentalStatus = 'Negative'
            else:
                sentimentalStatus = 'Neutural'
            cursor.execute("INSERT INTO SentimentAnalysisTable (UserID, Sentiment, Date) VALUES (%s, %s, %s);",
                           (ID, sentimentalStatus, Date))
            print("Inserted 1 row of data")
            print(tweet + " Sentiment Satus:" + sentimentalStatus)
            output = open("finalcoinSentiment.csv", "a", encoding='utf-8')
            output.write(Date+",")
            output.write(ID+",")
            output.write(sentimentalStatus+"\n")
            output.close()
            time.sleep(1)
            return True

    def on_error(self, status):
        print(status)


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["Bitcoin","Cryptocrunccy","Ethereum","Ripple","Blockchain","Tether"])



Connection established
Finished creating table
Inserted 1 row of data
Mit #ICOs können Unternehmen schnell viel #Kapital sammeln. Doch so flexibel die #Kryptowährungen sind, so groß sin… https://t.co/hu1iYhagZ3 Sentiment Satus:Negative
Inserted 1 row of data
https://t.co/M4XsdR5w3Y DashGol… https://t.co/hMM8xDGMFe Sentiment Satus:Negative
Inserted 1 row of data
Rémunérer les médias grâce au minage de crypto-monnaies plutôt qu'avec la publicité ? - - https://t.co/tdwN42XT7y… https://t.co/FDIb3tMIyb Sentiment Satus:Neutural
Inserted 1 row of data
Where @amitabhk87 acknowledges the extensive benefits of switching to alternate sources of meat on with savings on… https://t.co/1iVMIHEExU Sentiment Satus:Positive
Inserted 1 row of data
Copy successful traders automatically with Bitcoin! 💰💰

I copy "Dividend667". Do you?

➡️ https://t.co/RzkZmjw3m6… https://t.co/inzxYdyysL Sentiment Satus:Positive
Inserted 1 row of data
I don't know what @TheAbyss is developing using blockchain, but I will bec

Inserted 1 row of data
Major Banks Partner With R3 Blockchain Alliance To Launch Pilot Trade Platform - https://t.co/wWixF1dPDR Sentiment Satus:Neutural
Inserted 1 row of data
Cointelegraph #CT_quotes An Australian university has launched first #Blockchain course.
Developers see the strong… https://t.co/OTirfhsIHX Sentiment Satus:Positive
Inserted 1 row of data
“Vestarin” Platform berbasis BlockChain untuk Berbelanja dan Berdagang https://t.co/QDkgUUkxWu https://t.co/mXLM4sW8Oo Sentiment Satus:Neutural
Inserted 1 row of data
Cointelegraph Major exchanges Coinbase and Bifinex both announce SegWit support https://t.co/EUwka40XK0 Sentiment Satus:Positive
Inserted 1 row of data
Bitcoin Mining in 2018: How to Remain Profitable in Challenging Environment https://t.co/7Z11gfMO8H Sentiment Satus:Positive
Inserted 1 row of data
Cointelegraph #Japanese crypto exchange accidentally ‘sold’ $20 trln in #Bitcoin for 0 yen last week.
To read the f… https://t.co/FmYDSyDcMd Sentiment Satus:Negative
Ins

Inserted 1 row of data
Find more out about the FintruX Network Ecosysterm and FintruX customers in  whitepaper. 
https://t.co/i8NbGBLQCu …… https://t.co/pYNnHfjiDf Sentiment Satus:Neutural


ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [68]:
# Fetch all rows from table
cursor.execute("SELECT * FROM SentimentAnalysisTable;")
rows = cursor.fetchall()

In [69]:
# Print all rows
print(len(rows))
for row in rows:
    print(row[3])
    print("Data row = (%s, %s, %s, %s)" %(str(row[0]), str(row[1]), str(row[2]), str(row[3])))

100
Thu Feb 22 08:38:59 +0000 2018
Data row = (1, PE Magazin, Negative, Thu Feb 22 08:38:59 +0000 2018)
Thu Feb 22 08:39:00 +0000 2018
Data row = (2, Megalo Pump & Tools, Negative, Thu Feb 22 08:39:00 +0000 2018)
Thu Feb 22 08:39:02 +0000 2018
Data row = (3, eraser juⒶnjo * ✘ ★, Neutural, Thu Feb 22 08:39:02 +0000 2018)
Thu Feb 22 08:39:02 +0000 2018
Data row = (4, Harshdeep Singh, Positive, Thu Feb 22 08:39:02 +0000 2018)
Thu Feb 22 08:39:02 +0000 2018
Data row = (5, JosephLambert, Positive, Thu Feb 22 08:39:02 +0000 2018)
Thu Feb 22 08:39:03 +0000 2018
Data row = (6, I.T, Positive, Thu Feb 22 08:39:03 +0000 2018)
Thu Feb 22 08:39:03 +0000 2018
Data row = (7, ICO NEWS, Positive, Thu Feb 22 08:39:03 +0000 2018)
Thu Feb 22 08:39:04 +0000 2018
Data row = (8, Juanita Sainz, Neutural, Thu Feb 22 08:39:04 +0000 2018)
Thu Feb 22 08:39:04 +0000 2018
Data row = (9, RudiLo, Neutural, Thu Feb 22 08:39:04 +0000 2018)
Thu Feb 22 08:39:05 +0000 2018
Data row = (10, Matthew Okeke, Positive, Thu Feb 

In [70]:
# Cleanup
conn.commit()
cursor.close()
conn.close()